In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import sys
import numpy as np
import csv
import cv2
import argparse 
import imageio
from skimage.filters import threshold_otsu, threshold_mean, threshold_local, threshold_isodata
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects
from scipy.ndimage.morphology import binary_fill_holes
from skimage.morphology import square
from skimage import morphology
from skimage.segmentation import clear_border
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time



/tmp/ipykernel_426051/3803682953.py:15: DeprecationWarning: Please use `binary_fill_holes` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_fill_holes


In [2]:
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import binary_fill_holes
from skimage import morphology

def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert to Gray
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Gaussian Blur
    gray_blurred = cv2.GaussianBlur(gray, (5,5), 0)

    # Thresholding
    ret, thresh = cv2.threshold(gray_blurred, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Opening
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Dilation
    sure_bg = cv2.dilate(opening, kernel, iterations=2)

    # Distance Transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)

    # Thresholding for foreground
    ret, sure_fg = cv2.threshold(dist_transform, 0.3 * dist_transform.max(), 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Connected Components to differentiate regions
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    # Watershed
    markers = cv2.watershed(image, markers)

    # Convert markers to binary and fill holes
    bw = (markers > 1).astype(int)
    bw_filled = binary_fill_holes(bw).astype(np.uint8)

    # Remove small objects
    bw_clean = morphology.remove_small_objects(bw_filled.astype(bool), min_size=5000, connectivity=4).astype(np.uint8)

    # Save the final preprocessed image
    cv2.imwrite(image_path, bw_clean * 255)

def process_directory(src_directory):
    # Create a copy of the directory
    dst_directory = src_directory + "_copy"
    shutil.copytree(src_directory, dst_directory)

    # Iterate through the copied directory and preprocess .png files
    for root, _, files in os.walk(dst_directory):
        for filename in files:
            if filename.endswith('.png'):
                file_path = os.path.join(root, filename)
                preprocess_image(file_path)
                
    return dst_directory




In [3]:
# Test
src_directory = 'edof_new_sma_2023_cells_cleaned'
process_directory(src_directory)

'edof_new_sma_2023_cells_cleaned_copy'